In [20]:
import pandas as pd
import numpy as np
import requests
import python_files.pathsandmore as paths
import python_files.functions as f
import playsound
import time

from datetime import datetime 
from datetime import timedelta
import importlib

importlib.reload(f)
importlib.reload(paths)


print("DONE")

DONE


In [33]:
df=pd.read_csv(paths.pathnamelastupdate,
               compression='zip',
               parse_dates=['crash_date'],
               dtype = {'zip_code': 'str'},
               date_format="ISO8601"
              )
start_date=str(df.crash_date.max())[:10]
print("START DATE: ",start_date)
#Where to end:
final_date=input("Enter new date 'yyyy-mm-dd'")
#final_date='2024-12-05'
final_date_dt= datetime.strptime(final_date, '%Y-%m-%d')
start_date_dt= datetime.strptime(start_date, '%Y-%m-%d')
#start_date=start_date.to_pydatetime()
new_date=start_date_dt+timedelta(1)
#Creating df to store data
new_df=pd.DataFrame()
while new_date <= final_date_dt:
    new_date_str=str(new_date)[:10]
    url=f'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$$app_token={paths.app_Token}&crash_date={new_date_str}'
    resp=requests.get(url).json()
    print('RESP LEN:',len(resp),new_date_str)
    if new_df.empty:
        new_df=pd.DataFrame(resp)
        print(new_df.shape[0])
        new_date=new_date+timedelta(1)
    else:
        new_df=pd.concat([new_df,pd.DataFrame(resp)])
        print(new_date,new_df.shape[0])
        new_date=new_date+timedelta(1)

new_df=f.adding_shp_boro(new_df,paths.pathshapefileboro)
new_df=f.adding_shp_hood(new_df,paths.pathshapefilehood)

new_df=f.adding_shp_zip(new_df,paths.pathshapefilezipcode)

#eliminate columns duplicated with zip code
zip_columns=[col for col in new_df.columns if 'zip_code' in col]
new_df[zip_columns[0]]=new_df[zip_columns].bfill(axis=1).iloc[:,[0]]
new_df.drop(zip_columns[1:],axis=1,inplace=True)

new_df=f.adding_shp_st(new_df,paths.pathshapefilestreet)

#next step is combnine this with batches.
rename_columns={'vehicle_type_code1':'vehicle_type_code_1',
               'vehicle_type_code2':'vehicle_type_code_2'}
new_df.rename(columns=rename_columns,inplace=True)
new_df.reset_index(inplace=True,drop=True)
print(df.shape,new_df.shape)
final_df=pd.concat([df, new_df])
index_cols=[col for col in final_df.columns if col.startswith('index_')]
to_drop =index_cols + ['location', 'borough']
final_df.drop(to_drop,axis=1,inplace=True)
final_df.to_csv(paths.pathnamelastupdate,compression='zip',index=False)
f.play_done_message()

START DATE:  2025-06-06


Enter new date 'yyyy-mm-dd' 2025-6-08


RESP LEN: 165 2025-06-07
165
RESP LEN: 0 2025-06-08
2025-06-08 00:00:00 165
(2181393, 32) (165, 38)
